In [158]:
#export
from functools import partial, reduce
from operator import *
from typing import *
from pathlib import Path
import re, itertools, sys, operator

In [15]:
#export
Path.ls =  lambda x: list(x.iterdir)
def test_op(x, y, op:Callable):
    return eq(x, y)

In [12]:
#export
test_eq= partial(test_op,op=eq)
test_ne= partial(test_op, op=ne)
test_abs= partial(test_op, op=neg)

In [228]:
#export
def listify(x):
    if x is None: return []
    if isinstance(x, list): return x
    if isinstance(x, str): return [x]
    if isinstance(x, (Iterable, Generator)): return list(x)
    return [x]

In [229]:
test_eq(listify('narain'), ['narain'])
test_eq(listify({1,2,3,4,5}), list(range(1,6)))
test_eq(listify(range(5)), list(range(5)))
test_eq(listify((1,2,3)), [1,2,3])

True

In [230]:
#export
def setify(x): return set(listify(x))

In [231]:
#export
def is_indexer(idx):
    return isinstance(idx,int) or not getattr(idx,'ndim',1)

In [240]:
#export
def is_iter(o):
    "Test whether `o` can be used in a `for` loop"
    #Rank 0 tensors in PyTorch are not really iterable
    return isinstance(o, (Iterable,Generator))

In [233]:
is_iter([1,3])

True

In [244]:
#export
class L():
    def __init__(self, x):
        self.x= listify(x)
        
    def __len__(self): return len(self.x)
    def new(self, x): return type(self)(x) 
    
    def __getitem__(self, i): return self._get(i) if is_indexer(i) else L(self._get(i))
        
    def _get(self, i):
        if is_indexer(i) or isinstance(i,slice): return getattr(self.x,'iloc',self.x)[i]
        i = mask2idxs(i)
        return (self.x.iloc[list(i)] if hasattr(self.x,'iloc')
                else self.x.__array__()[(i,)] if hasattr(self.x,'__array__')
                else [self.x[i_] for i_ in i])
    
    def __setitem__(self, i, o):
        i = i if is_iter(i) else listify(i)
        if not is_iter(o): o = [o]*len(i)
        print(i)
        for i_,o_ in zip(i,o):
            self.x[i_] = o_
    
    def __repr__(self):
        return f'(#{len(self.x)}) [' + ','.join(itertools.islice(map(repr,self.x), 10)) + (
        '...' if len(self.x)>10 else '') + ']'
    
    def __iter__(self): return iter(self.x.itertuples() if hasattr(self.x,'iloc') else self.x)
    def __contains__(self,b): return b in self.x
    def __invert__(self): return self.new(not i for i in self)
    def __eq__(self,b): return False if isinstance(b, (str,dict,set)) else all_equal(b,self)
    def __mul__ (a,b): return a._new(a.items*b)
    def __add__ (a,b): return a._new(a.items+_listify(b))
    def __radd__(a,b): return a._new(b)+a
    def __addi__(a,b):
        a.items += list(b)
        return a

    def append(self,o): return self.x.append(o)
    def remove(self,o): return self.x.remove(o)
    def count (self,o): return self.x.count(o)
    def reverse(self ): return self.x.reverse()
    def pop(self,o=-1): return self.x.pop(o)
    def clear(self   ): return self.x.clear()
    def index(self, value, start=0, stop=sys.maxsize): return self.x.index(value, start=start, stop=stop)
    def sort(self, key=None, reverse=False): return self.x.sort(key=key, reverse=reverse)
    def reduce(self, f, initial=None): return reduce(f, self) if initial is None else reduce(f, self, initial)
    def sum(self): return self.reduce(operator.add)
    def product(self): return self.reduce(operator.mul)

In [245]:
l=L(range(11)); l

(#11) [0,1,2,3,4,5,6,7,8,9...]

In [246]:
l[1:7]=0

[slice(1, 7, None)]


TypeError: can only assign an iterable

In [237]:
L([1,2,3,4,5,6,7,8,9,1,2,3,4,5])

(#14) [1,2,3,4,5,6,7,8,9,1...]

In [238]:
L([1,2,3,4,5])

(#5) [1,2,3,4,5]